# **First Project: Equal-Weight S&P 500 Stock Screener**

- In the first project I build an alternative version of the S&P 500 index where each company has the same weighting.

### **Introduction & Library Imports**

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust.

The goal of this project is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

In [2]:
# Importing Libraries
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import yfinance as yf
from datetime import datetime, timedelta

### **Importing Our List of Stocks**

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

In [4]:
# Stocks to be studied (this my be older version of included stocks in S&P 500)
stock_names = pd.read_csv('sp_500_stocks.csv')

### **Create a new DataFrame including market cap, price and number of Stocks columns**

In [16]:
my_columns = ['Ticker', 'Price', 'Market Capitalization', 'Number Of Shares to Buy']

new_data = pd.DataFrame(columns=my_columns)

for ticker in stock_names['Ticker']:
    stock = yf.Ticker(ticker)
    info = stock.info

    price = info.get('currentPrice', info.get('regularMarketPrice', None))

    new_row = pd.DataFrame([{
        'Ticker': ticker,
        'Price': price,
        'Market Capitalization': info.get('marketCap', None),
        'Number Of Shares to Buy': 0  # Initialized as 0
    }])

    new_data = pd.concat([new_data, new_row], ignore_index=True)

C:\Users\Altair\AppData\Local\Temp\ipykernel_13164\1057041937.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_data = pd.concat([new_data, new_row], ignore_index=True)
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=WeUgHT9P02G
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=WeUgHT9P02G
404 Client Error: Not Found for url: https://query

### **Calculating the Numver of Shares to BUY**

In [2]:
final_dataframe = new_data

# Enter portofolio size in dollars
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
    print(f"Portofolio size in euros : {portfolio_size}")
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")
    
# Remove each stocks that is no longer available in the Exchange
final_dataframe = final_dataframe.dropna(subset=['Price'])

# Calculate the number of shared for an equal weighted protofolio

final_dataframe.reset_index(drop=True, inplace=True)
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

NameError: name 'new_data' is not defined

**Export csv data :** Using XlsxWriter we are going to save our equal - weighted portofolio into an excel file (instead of putting real trade orders)

In [1]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

writer.close()

NameError: name 'pd' is not defined